In [ ]:
# Cell 1: INSTRUCTIONS

''' 
This code gathers Private Holdings for Mutual Funds from Filing Data Links
Please Follow the Following Instructions

Step 1: Run Cell 2. This Cell will import all necessary python modules to run the script. 
        It it possible that not all of these modules are installed on your machine. 
        To install these modules, open the terminal and type "pip install insert_module_here". 
        The pip install commands for each module are printed next to each import

        For example, to install BeautifulSoup, open the terminal and enter "pip install BeautifulSoup4"ArithmeticError

Step 2: Run Cell 3. This will save all of the necessary functions needed to gather filing data.

Step 3: Save the the desired list of URLs to the variable "URL_list." 
        There are 2 methods for doing this

        Method 1: Manually enter a list of URLs (Follow Instructions in Cell 4)
        Method 2: Load in a list of URLs from a csv file (Follow Instructions in Cell 5)

Step 4: Follow Instructions in Cell 6 and then run Cell 6. 
        This will save and download all holding information to a csv file.
'''


In [1]:
# Cell 2: Imports

from bs4 import BeautifulSoup # pip install BeautifulSoup4
import requests # pip install requests
import pandas as pd # pip install pandas
pd.set_option('display.float_format', '{:.2f}'.format)
import time 
from tqdm.notebook import tqdm # pip install tqdm

In [2]:
# Cell 3: Functions

def Request(URL, Type):
    """
    Function: Get XML or HTML Soup Object

    Variables
        URL: XML URL
        Type: Request Type (XML or HTML)
    """

    error = False # Used for troubleshooting

    while True:
        request = requests.get(URL, headers={'User-Agent': 'last_name.first_name@outlook.com'})

        if request.status_code == 200:
            error = False
            if Type == 'XML':
                soup = BeautifulSoup(request.text, "xml")
            elif Type == 'HTML':
                soup = BeautifulSoup(request.content, 'html.parser')
            return soup
        
        else:
            if error == False:
                error = True
                time.sleep(5) # pause code for 5 seconds to see if request error goes away
            elif error == True:
                print(f"Received unexpected status code {request.status_code}")
                return None

def primary_doc(URL):
    """
    Function: Grab Primary Doc File From Filing Detail Link

    Variables
        URL: Filing Detail Link
    """
    soup = Request(URL, 'HTML')

    doc = Request(f"https://www.sec.gov{soup.select('.blueRow a')[0]['href']}", 'XML') # Grab XML data
    period_of_report = soup.select(".formGrouping+ .formGrouping .info")[0].text # Grab Filing Date and Reporting Date
    filing_date = soup.select(".formGrouping:nth-child(1) .info:nth-child(2)")[0].text

    return doc, period_of_report, filing_date, URL

def registrant_info(soup):

    """
    Function: Generate Series Containing Registrant Information

    Variables
        soup: XML Soup Object
    """

    reg_info = soup.find_all("genInfo")[0].find_all(True) # Grab all Registrant Info from soup object
    reg_info_series = pd.Series({tag.name: tag.text for tag in reg_info}) # Format into pandas series

    return reg_info_series

def investment_info(investment):

    """
    Function: Generate Dictionary Containing Portfolio Restricted Holding Information

    Variables
        investment: A single 'invstOrSec' portion of XML file
    """

    tags = investment.find_all(True) # Grab all tags from holding
    tag_d = {tag.name: tag.text for tag in tags} # Format into a dictionary

    if tag_d['isRestrictedSec'] == 'Y': # Only perform code on Restriced Securities
        return_d = {}
        return_d['Issuer'] = tag_d['name']
        return_d['Title'] = tag_d['title']
        return_d['Current_Balance'] = float(tag_d['balance'])

        if tag_d['valUSD'] != 'N/A':
            return_d['Total_Value'] = float(tag_d['valUSD'])
        else:
            return_d['Total_Value'] = 0

        try:
            return_d['Asset_Class'] = tag_d['assetCat']
        except: 
            return_d['Asset_Class'] = 'Other'

        return_d['Payoff_Profile'] = tag_d['payoffProfile']
        return_d['Fair_Value_Level'] = tag_d['fairValLevel']
        return_d['Restricted'] = tag_d['isRestrictedSec']

        return return_d
    
    else:
        return None

def format_df(URL):

    """
    Function:
        Format XML of holdings information into DataFrame

    Variables
        doc: Primary Doc Tuple Output (Contains XML Soup Object, Reporting and Filing Date, and Original URL)
    """

    soup, period_of_report, filing_date, URL = primary_doc(URL)
    
    col_order = ['Form_Type', 'Company_Name','CIK','Fund_Series','Series_ID','Reporting_Date',
                     'Filing_Date','Issuer', 'Title','Valuation','Current_Balance','Total_Value',
                     'Asset_Class','Payoff_Profile','Fair_Value_Level','Link','Restricted']
    
    portfolio_columns = ['Issuer','Title','Current_Balance', 'Total_Value', 
                         'Asset_Class','Payoff_Profile', 'Fair_Value_Level','Restricted']

    reg_info = registrant_info(soup) # Get Registrant Info

    investments = [investment_info(investment) for investment in soup.find_all("invstOrSec")] # Get tag information for each restriced holding
    
    portfolio_d = {column : [] for column in portfolio_columns} # Dictionary to later be turned into a dataframe
    
    for d in investments: # Loop through each investment
        for column in portfolio_columns:
            try:
                portfolio_d[column].append(d[column]) # If the investment has a tag in portfolio_columns, add it to the dictionary
            except:
                portfolio_d[column].append(None) # Otherwise, leave this field blank

    return_df = pd.DataFrame(portfolio_d).assign(Form_Type = soup.headerData.submissionType.text,
                                                 Company_Name = reg_info.regName, # Format into a dataframe and assign Registrant Info
                                                 CIK = reg_info.regCik,
                                                 Fund_Series = reg_info.seriesName,
                                                 Reporting_Date = period_of_report,
                                                 Filing_Date = filing_date,
                                                 Valuation = lambda df: df['Total_Value']/df['Current_Balance'],
                                                 Link = URL)

    if 'seriesId' in reg_info:
        return_df = return_df.assign(Series_ID = reg_info.seriesId)
    else:
        return_df = return_df.assign(Series_ID = 'N/A')

    return_df = return_df.loc[return_df.Restricted == 'Y',col_order].drop(columns=['Restricted']) # Reorder Columns

    return_df['Reporting_Date'] = pd.to_datetime(return_df['Reporting_Date']) # Convert to datetime datatype
    return_df['Filing_Date'] = pd.to_datetime(return_df['Filing_Date'])

    return return_df

def main(URL_List, csv_name, save_csv = False, google_colab = False):

    URL_List = tqdm(URL_List, desc="Gathering Data") # Format to show progress bar

    final_df = (pd.concat(list(map(format_df, URL_List))) # Loop through each URL and stack outputs
                    .sort_values(['Company_Name','Title','Reporting_Date','Fund_Series']) # Reorder Rows
                    .reset_index(drop=True))
    
    if save_csv == True:
        final_df.to_csv(f"{csv_name}.csv")

        if google_colab == True:
            from google.colab import files
            files.download(f"{csv_name}.csv") 


    return final_df

In [3]:
# Cell 4: Manual Links

'''
Instructions for Manual URLs

Step 1: Enter each link in quotes as a comma seperated list between brackets 
  Note: If there is only 1 URL, it still must be surrounded by brackets

Step 2: Run this Cell to assign the list of links to the variable URL_list
  Note: Do not run the Upload Links Cell. This will override the URL List assigned here

Step 3: Run the Main Function Below to gather data from links
'''


URL_list = ['https://www.sec.gov/Archives/edgar/data/1319067/0001387131-21-001485-index.htm',
           'https://www.sec.gov/Archives/edgar/data/1319067/0001387131-21-002834-index.htm',
           'https://www.sec.gov/Archives/edgar/data/1319067/0001387131-21-002914-index.htm']





In [5]:
# Cell 5: Upload Links From Spreadsheet

'''
Step 1: Set ```path``` equal to the file path (in quotations) for the csv file where the links are located.
        This will assign the file path to the "path" variable
  Note: If running in google colab, first upload the csv file by clicking the files
        icon on the left side and clicking "upload to session storage" icon near the top

Step 2: Set ```column_name``` to the name of the column (in quotations) where the 
        links are located inside the spreadsheet.

Step 3: Run this Cell to assign the list of links to the variable URL_list
  Note: Do not run the Upload Links Cell. This will override the URL List assigned here

Step 4: Run Cell 6 to gather data from links
'''


path = '2021-2023 NPORT filings.csv'
column_name = 'File_Name'

csv = pd.read_csv(path)
URL_list = csv.loc[150000:,column_name] 


In [1]:
# Cell 6: Main Function

'''
Saving as a csv file

csv_name: set equal to whatever you would like the resulting csv to be called (in quotations).
save_csv: set equal to True (no quotations) to save csv, otherwise set to False (no quotations)
google_colab: If using Google Colab, set equal to True (no quotations), otherwise set to False (no quotations)
'''

csv_name = 'my_csv'
save_csv = False
google_colab = False

df = main(URL_list, csv_name = csv_name, save_csv = save_csv, google_colab = google_colab)

df

NameError: name 'main' is not defined